In [3]:
import numpy as np

X = []
Y = []

input_noises = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
model_name = "cam"

for input_noise in input_noises:
    print(f"Adding entries with {input_noise=}")
    with open(f"../utils/datasets/{model_name}.csv") as f:
        for i, line in enumerate(f):
            data = line.split(",")
            x = np.array(data[2:], dtype=np.float32)
            y = np.array(data[:2], dtype=np.float32)

            y = y[1]  # steer only

            # if adding noise, redo multiple times to reduce overfitting to noise
            if input_noise > 0:
                for cell in range(len(x)):
                    if np.random.rand() < input_noise:
                        x[cell] = 0 if x[cell] == 1 else 1  # random bit flips

            grid_size = 30
            x = x.reshape(grid_size, grid_size, 1)  # transform state back to grid

            X.append(x)
            Y.append(y)

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

print(f"Loaded {len(X)} entires")
print(f"entry {x.shape=}, {y.shape=}")

Adding entries with input_noise=0.0
Adding entries with input_noise=0.1
Adding entries with input_noise=0.2
Adding entries with input_noise=0.3
Adding entries with input_noise=0.4
Adding entries with input_noise=0.5
Loaded 82752 entires
entry x.shape=(30, 30, 1), y.shape=()


In [14]:
import keras
print(keras.__version__)

2.13.1


In [13]:
import keras

model = keras.models.Sequential(
    [
        # lenet-5
        keras.layers.Conv2D(
            16, kernel_size=(5, 5), activation="relu", input_shape=(30, 30, 1)
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(120, activation="relu"),
        keras.layers.Dense(84, activation="relu"),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

# TODO: add normal following to all experts' ds

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
)

model.fit(
    X,
    Y,
    epochs=5,
    # validation_split=0.2,
    # callbacks=[
    #     keras.callbacks.EarlyStopping(patience=1),
    # ],
)

model.save(f"../controllers/moe_controller/models/{model_name}.keras")

Epoch 1/5
2586/2586 [==============================] - 14s 5ms/step - loss: 0.0177
Epoch 2/5
2586/2586 [==============================] - 13s 5ms/step - loss: 0.0159
Epoch 3/5
2586/2586 [==============================] - 13s 5ms/step - loss: 0.0151
Epoch 4/5
2586/2586 [==============================] - 13s 5ms/step - loss: 0.0145
Epoch 5/5
2586/2586 [==============================] - 13s 5ms/step - loss: 0.0140


In [28]:
# time inference speed
import time

start = time.time()
model(X[0:100])
end = time.time()
avg = (end - start) / 100
print(f"avg inference time: {avg*1000:.2f}ms")

avg inference time: 0.11ms
